In [10]:
# load in libraries
import pandas as pd
import os
import folium

In [11]:
# Read in processed airports data
airports_dir = os.path.join(os.getcwd(), '../data/raw_data/airports_processed.csv')
airports = pd.read_csv(airports_dir)

In [12]:
# Drop rows with missing coordinates
airports = airports.dropna(subset=['latitude_deg', 'longitude_deg'])

# Create interactive map centered over the continental US
m = folium.Map(location=[39.5, -98.35], zoom_start=4)

# Add airport points with all column data in popup
for _, row in airports.iterrows():
    popup_html = "<br>".join([f"<b>{col}:</b> {row[col]}" for col in airports.columns])
    folium.CircleMarker(
        location=[row['latitude_deg'], row['longitude_deg']],
        radius=5,
        color='blue',
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(popup_html, max_width=300),
        tooltip=row['name'] if 'name' in row else 'Airport'
    ).add_to(m)

# Save the interactive map to an HTML file
m.save('interactive_airport_map.html')

In [13]:
# display
m